## Homework: Multilingual Embedding-based Machine Translation (7 points)

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully). 

For our system we choose two kindred Slavic languages: Ukrainian and Russian. 

### Feel the difference!

(_синій кіт_ vs. _синій кит_)

![blue_cat_blue_whale.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/blue_cat_blue_whale.png)

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [1]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

Download embeddings here:
* [cc.uk.300.vec.zip](https://yadi.sk/d/9CAeNsJiInoyUA)
* [cc.ru.300.vec.zip](https://yadi.sk/d/3yG0-M4M8fypeQ)

Load embeddings for ukrainian and russian.

In [4]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [5]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

In [6]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

/Users/kowalski/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('август', 1.0),
 ('июль', 0.9383153319358826),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222576022148132),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930035829544067),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796871185303),
 ('февраль', 0.8401416540145874)]

In [7]:
uk_emb.most_similar([uk_emb["серпень"]])

/Users/kowalski/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('серпень', 1.0),
 ('липень', 0.9096439480781555),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992519974708557),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431221961975)]

In [8]:
ru_emb.most_similar([uk_emb["серпень"]])

/Users/kowalski/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Недопустимость', 0.24435287714004517),
 ('конструктивность', 0.23293082416057587),
 ('офор', 0.23256802558898926),
 ('deteydlya', 0.23031717538833618),
 ('пресечении', 0.22632381319999695),
 ('одностороннего', 0.22608885169029236),
 ('подход', 0.2230587601661682),
 ('иболее', 0.22003725171089172),
 ('2015Александр', 0.21872764825820923),
 ('конструктивен', 0.21796566247940063)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [9]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [10]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [11]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

In Greek mythology, Procrustes or "the stretcher" was a rogue smith and bandit from Attica who attacked people by stretching them or cutting off their legs, so as to force them to fit the size of an iron bed. We make same bad things with source embedding space. Our Procrustean bed is target embedding space.

![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

![procrustes.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/procrustes.png)

But wait...$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [12]:
from sklearn.linear_model import LinearRegression

mapping = LinearRegression()
mapping.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [13]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

/Users/kowalski/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('апрель', 0.8541592359542847),
 ('июнь', 0.8411963582038879),
 ('март', 0.8397400379180908),
 ('сентябрь', 0.835921585559845),
 ('февраль', 0.8328748941421509),
 ('октябрь', 0.8311805725097656),
 ('ноябрь', 0.8278146982192993),
 ('июль', 0.8236351013183594),
 ('август', 0.8120611906051636),
 ('декабрь', 0.8037999868392944)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [41]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        print(ru, ru_emb.most_similar(mapped_vectors))
        for word, _ in ru_emb.most_similar(mapped_vectors)[:topn]:
            if ru == word:
                num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val


In [39]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

/Users/kowalski/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


август [('апрель', 0.8541592359542847), ('июнь', 0.8411963582038879), ('март', 0.8397400379180908), ('сентябрь', 0.835921585559845), ('февраль', 0.8328748941421509)]
август [('апрель', 0.8541592359542847), ('июнь', 0.8411963582038879), ('март', 0.8397400379180908), ('сентябрь', 0.835921585559845), ('февраль', 0.8328748941421509), ('октябрь', 0.8311805725097656), ('ноябрь', 0.8278146982192993), ('июль', 0.8236351013183594), ('август', 0.8120611906051636)]
август [('апрель', 0.8541592359542847), ('июнь', 0.8411963582038879), ('март', 0.8397400379180908), ('сентябрь', 0.835921585559845), ('февраль', 0.8328748941421509), ('октябрь', 0.8311805725097656), ('ноябрь', 0.8278146982192993), ('июль', 0.8236351013183594), ('август', 0.8120611906051636), ('декабрь', 0.8037999868392944)]


In [42]:
print(precision(uk_ru_test, Y_test))

/Users/kowalski/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


либо [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
активный [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
актер [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
ж [('это', 0.6054540276527405), ('э

выступить [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
отвести [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
открываться [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
отличный [('это', 0.60545

гражданин [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
декабрь [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
почва [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
крыша [('это', 0.60545402765274

экспедиция [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
электрический [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
элемент [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
сцена [('это', 0.60545

...

скрывать [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
проект [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
об [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
однако [('это', 0.6054540276527405),

сырой [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
семьи [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
семьсот [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
скромный [('это', 0.605454027652740

судья [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
печальный [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
сплошной [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
ступенька [('это', 0.605454027

наряд [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
вечер [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
украсть [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
университет [('это', 0.605454027652

сапоги [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
мужчина [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
иностранец [('это', 0.6054540276527405), ('этот', 0.5874260663986206), ('конечно', 0.5778403282165527), ('все-таки', 0.5725287199020386), ('только', 0.5686001181602478), ('наверное', 0.5645053386688232), ('вообще', 0.561825692653656), ('опять', 0.5587222576141357), ('даже', 0.5572904944419861), ('ведь', 0.5555437803268433)]
слышать [('это', 0.6054540276

In [37]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

/Users/kowalski/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


AssertionError: 

In [43]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

/Users/kowalski/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


либо [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
активный [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
актер [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
ж [('это', 0.6189959049224854), ('т

приговор [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
выступить [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
отвести [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
открываться [('это', 0.61899

готовность [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
гражданин [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
декабрь [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
почва [('это', 0.618995904

духовный [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
экспедиция [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
электрический [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
элемент [('это', 0.61

...

реки [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
речка [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
разглядывать [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
полотенце [('это', 0.6189959049

серебряный [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
курс [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
старинный [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
создавать [('это', 0.61899590

май [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
травы [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
традиция [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
случиться [('это', 0.618995904922485

крест [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
сей [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
церковь [('это', 0.6189959049224854), ('только', 0.5936696529388428), ('конечно', 0.592339277267456), ('наверное', 0.5858299732208252), ('даже', 0.5844234824180603), ('все-таки', 0.582939624786377), ('этот', 0.5824849009513855), ('вообще', 0.5793285369873047), ('просто', 0.5739447474479675), ('ведь', 0.5701208114624023)]
гражданский [('это', 0.61899590492248

AssertionError: 

## Making it better (orthogonal Procrustean problem)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [140]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOU CODE HERE

In [141]:
W = learn_transform(X_train, Y_train)

In [ ]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

In [ ]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

## UK-RU Translator

Now we are ready to make simple word-based translator: for earch word in source language in shared embedding space we find the nearest in target language.


In [186]:
with open("fairy_tale.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [187]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    # YOUR CODE HERE

In [ ]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [ ]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

Not so bad, right? We can easily improve translation using language model and not one but several nearest neighbours in shared embedding space. But next time.

## Would you like to learn more?

### Articles:
* [Exploiting Similarities among Languages for Machine Translation](https://arxiv.org/pdf/1309.4168)  - entry point for multilingual embedding studies by Tomas Mikolov (the author of W2V)
* [Offline bilingual word vectors, orthogonal transformations and the inverted softmax](https://arxiv.org/pdf/1702.03859) - orthogonal transform for unsupervised MT
* [Word Translation Without Parallel Data](https://arxiv.org/pdf/1710.04087)
* [Loss in Translation: Learning Bilingual Word Mapping with a Retrieval Criterion](https://arxiv.org/pdf/1804.07745)
* [Unsupervised Alignment of Embeddings with Wasserstein Procrustes](https://arxiv.org/pdf/1805.11222)

### Repos (with ready-to-use multilingual embeddings):
* https://github.com/facebookresearch/MUSE

* https://github.com/Babylonpartners/fastText_multilingual -